In [3]:
import pandas as pd
import numpy as np

In [4]:
#Training
df = pd.read_excel ('exams.xlsx')
df

,gender,race/ethnicity,parental level of education,Class_lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associate's degree,standard,completed,78,73,68
...,...,...,...,...,...,...,...,...
995,male,group C,high school,standard,none,73,70,65
996,male,group D,associate's degree,free/reduced,completed,85,91,92
997,female,group C,some high school,free/reduced,none,32,35,41
998,female,group C,some college,standard,none,73,74,82


In [5]:
df.head()

,gender,race/ethnicity,parental level of education,Class_lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associate's degree,standard,completed,78,73,68


In [10]:
cats = [
    {'name' : 'math score','bins' : [0, 40 ,50, 65, 80, 100], 
     'labels' : ['E', 'D', 'C', 'B', 'A']},
    {'name' : 'reading score','bins' : [0, 40 ,50, 65, 80, 100], 
     'labels' : ['E', 'D', 'C', 'B', 'A']},
    {'name' : 'writing score','bins' : [0, 40 ,50, 65, 80, 100], 
      'labels' : ['E', 'D', 'C', 'B', 'A']},
]

In [11]:
def to_categorical(data):
  for cat in cats:
    col = cat['name']
    bins = cat['bins']
    labels = cat ['labels']
    data[col] = pd.cut(data[col], bins=bins, labels=labels, include_lowest = True)

to_categorical(df)

In [12]:
df

,gender,race/ethnicity,parental level of education,Class_lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,B,B,C
1,female,group D,some high school,free/reduced,none,E,C,C
2,male,group E,some college,free/reduced,none,C,C,D
3,male,group B,high school,standard,none,B,B,B
4,male,group E,associate's degree,standard,completed,B,B,B
...,...,...,...,...,...,...,...,...
995,male,group C,high school,standard,none,B,B,C
996,male,group D,associate's degree,free/reduced,completed,A,A,A
997,female,group C,some high school,free/reduced,none,E,E,D
998,female,group C,some college,standard,none,B,B,A


In [13]:
total_rows = len(df)
total_rows

y_col = 'Class_lunch'
y_labels = df[y_col].unique()
y_labels_count = df[y_col].value_counts()
y_labels

array(['standard', 'free/reduced'], dtype=object)

In [19]:
lookup_table = {}
for i, row in df.iterrows():
  for col in cats:
    col_name = col['name']
    col_class = row[col_name]
    for y in y_labels:
      logic = (df[col_name]==col_class) & (df[y_col]==y)
      locator = y + col_name + col_class
      lookup_table[locator] = len(df[logic]) / y_labels_count[y]

for keys, val in lookup_table.items():
  print(repr(keys), ' : ', round(val,2))

'standardmath scoreB'  :  0.39
'free/reducedmath scoreB'  :  0.25
'standardreading scoreB'  :  0.39
'free/reducedreading scoreB'  :  0.31
'standardwriting scoreC'  :  0.27
'free/reducedwriting scoreC'  :  0.33
'standardmath scoreE'  :  0.02
'free/reducedmath scoreE'  :  0.12
'standardreading scoreC'  :  0.27
'free/reducedreading scoreC'  :  0.35
'standardmath scoreC'  :  0.29
'free/reducedmath scoreC'  :  0.38
'standardwriting scoreD'  :  0.08
'free/reducedwriting scoreD'  :  0.17
'standardwriting scoreB'  :  0.36
'free/reducedwriting scoreB'  :  0.28
'standardmath scoreA'  :  0.25
'free/reducedmath scoreA'  :  0.08
'standardreading scoreA'  :  0.28
'free/reducedreading scoreA'  :  0.13
'standardwriting scoreA'  :  0.28
'free/reducedwriting scoreA'  :  0.12
'standardmath scoreD'  :  0.07
'free/reducedmath scoreD'  :  0.18
'standardreading scoreD'  :  0.05
'free/reducedreading scoreD'  :  0.14
'standardreading scoreE'  :  0.01
'free/reducedreading scoreE'  :  0.07
'standardwriting score

In [21]:
for i, row in df.iterrows():
  probs = []
  for y in y_labels:
    py = 1
    for col in cats:
      col_name = col['name']
      col_class = row[col_name]
      locator = y + col_name + col_class
      value = lookup_table[locator]
      py *= value
    probs.append(py)

  #klasifikasi
  iMax = np.argmax(probs)
  valMax = np.max(probs)
  prediction = y_labels[iMax]
  df[y_col][i] = prediction
  print(probs, 'OUTPUT: ', prediction, '({})'.format(valMax))

<ipython-input-21-6490feeef209>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[y_col][i] = prediction


[0.041268428980035084, 0.02510974703228383] OUTPUT:  standard (0.041268428980035084)
[0.0011174457893753662, 0.013760970906738466] OUTPUT:  free/reduced (0.013760970906738466)
[0.005987123872625209, 0.022729680047015727] OUTPUT:  free/reduced (0.022729680047015727)
[0.05378806473802326, 0.02161621701040086] OUTPUT:  standard (0.05378806473802326)
[0.05378806473802326, 0.02161621701040086] OUTPUT:  standard (0.05378806473802326)
[0.03991415915083472, 0.03317837959735946] OUTPUT:  standard (0.03991415915083472)
[0.020896236261319354, 0.04430361365096286] OUTPUT:  free/reduced (0.04430361365096286)
[0.01943775519558173, 0.001227784839249024] OUTPUT:  standard (0.01943775519558173)
[0.020896236261319354, 0.04430361365096286] OUTPUT:  free/reduced (0.04430361365096286)
[0.0002531895767635468, 0.004099070163689459] OUTPUT:  free/reduced (0.004099070163689459)
[0.01943775519558173, 0.001227784839249024] OUTPUT:  standard (0.01943775519558173)
[0.030614464433041225, 0.0037710534348362884] OUTP

In [22]:
df

,gender,race/ethnicity,parental level of education,Class_lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,B,B,C
1,female,group D,some high school,free/reduced,none,E,C,C
2,male,group E,some college,free/reduced,none,C,C,D
3,male,group B,high school,standard,none,B,B,B
4,male,group E,associate's degree,standard,completed,B,B,B
...,...,...,...,...,...,...,...,...
995,male,group C,high school,standard,none,B,B,C
996,male,group D,associate's degree,standard,completed,A,A,A
997,female,group C,some high school,free/reduced,none,E,E,D
998,female,group C,some college,standard,none,B,B,A
